In [1]:
import os
from glob import glob # extract path of each file
import pandas as pd # data preprocessing
from xml.etree import ElementTree as et # parse information from XML
from functools import reduce

In [2]:
# step-1: get path of each xml file
xmlfiles = glob('./data_images/*.xml')
# replace \\ with /
# replace_text = lambda x: x.replace('\\','/')
# xmlfiles = list(map(replace_text,xmlfiles))

In [3]:
xmlfiles

['./data_images/007826.xml',
 './data_images/002786.xml',
 './data_images/006286.xml',
 './data_images/002962.xml',
 './data_images/008297.xml',
 './data_images/009189.xml',
 './data_images/009823.xml',
 './data_images/002976.xml',
 './data_images/002745.xml',
 './data_images/006523.xml',
 './data_images/008268.xml',
 './data_images/004452.xml',
 './data_images/002023.xml',
 './data_images/005980.xml',
 './data_images/004446.xml',
 './data_images/002037.xml',
 './data_images/009162.xml',
 './data_images/006251.xml',
 './data_images/000620.xml',
 './data_images/000146.xml',
 './data_images/007629.xml',
 './data_images/001258.xml',
 './data_images/002751.xml',
 './data_images/002989.xml',
 './data_images/007601.xml',
 './data_images/001270.xml',
 './data_images/002779.xml',
 './data_images/005016.xml',
 './data_images/003301.xml',
 './data_images/006279.xml',
 './data_images/007167.xml',
 './data_images/008254.xml',
 './data_images/000608.xml',
 './data_images/005764.xml',
 './data_image

In [4]:
# filename, size(width, height), object(name, xmin, xmax, ymin, ymax)
def extract_text(filename):
    tree = et.parse(filename)
    root = tree.getroot()

    # extract filename
    image_name = root.find('filename').text
    # width and height of the image
    width = root.find('size').find('width').text
    height = root.find('size').find('height').text
    objs = root.findall('object')
    parser = []
    for obj in objs:
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = bndbox.find('xmin').text
        xmax = bndbox.find('xmax').text
        ymin = bndbox.find('ymin').text
        ymax = bndbox.find('ymax').text
        parser.append([image_name, width, height, name,xmin,xmax,ymin,ymax])
        
    return parser

In [6]:
parser_all = list(map(extract_text,xmlfiles))

In [8]:
data = reduce(lambda x, y : x+y,parser_all)

In [10]:
df = pd.DataFrame(data,columns = ['filename','width','height','name','xmin','xmax','ymin','ymax'])

In [12]:
df['name'].value_counts()

person         5447
car            1650
chair          1427
bottle          634
pottedplant     625
bird            599
dog             538
sofa            425
bicycle         418
horse           406
boat            398
motorbike       390
cat             389
tvmonitor       367
cow             356
sheep           353
aeroplane       331
train           328
diningtable     310
bus             272
Name: name, dtype: int64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15663 entries, 0 to 15662
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  15663 non-null  object
 1   width     15663 non-null  object
 2   height    15663 non-null  object
 3   name      15663 non-null  object
 4   xmin      15663 non-null  object
 5   xmax      15663 non-null  object
 6   ymin      15663 non-null  object
 7   ymax      15663 non-null  object
dtypes: object(8)
memory usage: 979.1+ KB


In [14]:
cols = ['width','height','xmin','xmax','ymin','ymax']
df[cols] = df[cols].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15663 entries, 0 to 15662
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  15663 non-null  object
 1   width     15663 non-null  int64 
 2   height    15663 non-null  int64 
 3   name      15663 non-null  object
 4   xmin      15663 non-null  int64 
 5   xmax      15663 non-null  int64 
 6   ymin      15663 non-null  int64 
 7   ymax      15663 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 979.1+ KB


In [17]:
# center x, center y
df['center_x'] = ((df['xmax']+df['xmin'])/2)/df['width']
df['center_y'] = ((df['ymax']+df['ymin'])/2)/df['height']
# w 
df['w'] = (df['xmax']-df['xmin'])/df['width']
# h 
df['h'] = (df['ymax']-df['ymin'])/df['height']
df

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,007826.jpg,500,375,diningtable,80,320,217,273,0.400000,0.653333,0.480000,0.149333
1,007826.jpg,500,375,chair,197,257,193,326,0.454000,0.692000,0.120000,0.354667
2,007826.jpg,500,375,chair,139,185,184,231,0.324000,0.553333,0.092000,0.125333
3,007826.jpg,500,375,chair,258,312,180,314,0.570000,0.658667,0.108000,0.357333
4,007826.jpg,500,375,chair,10,93,195,358,0.103000,0.737333,0.166000,0.434667
...,...,...,...,...,...,...,...,...,...,...,...,...
15658,004488.jpg,500,375,car,324,368,239,267,0.692000,0.674667,0.088000,0.074667
15659,005796.jpg,375,500,person,45,294,57,500,0.452000,0.557000,0.664000,0.886000
15660,005796.jpg,375,500,tvmonitor,300,375,275,387,0.900000,0.662000,0.200000,0.224000
15661,002947.jpg,375,500,sofa,6,375,291,500,0.508000,0.791000,0.984000,0.418000


In [18]:
##split 

In [19]:
images = df['filename'].unique()

In [20]:
img_df = pd.DataFrame(images,columns=['filename'])
img_train = tuple(img_df.sample(frac=0.8)['filename'])


In [21]:
img_test = tuple(img_df.query(f'filename not in {img_train}')['filename']) # take rest 20% images

In [22]:
len(img_train), len(img_test)

(4010, 1002)

In [23]:
train_df = df.query(f'filename in {img_train}')
test_df = df.query(f'filename in {img_test}')

In [24]:
train_df

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,007826.jpg,500,375,diningtable,80,320,217,273,0.400000,0.653333,0.480000,0.149333
1,007826.jpg,500,375,chair,197,257,193,326,0.454000,0.692000,0.120000,0.354667
2,007826.jpg,500,375,chair,139,185,184,231,0.324000,0.553333,0.092000,0.125333
3,007826.jpg,500,375,chair,258,312,180,314,0.570000,0.658667,0.108000,0.357333
4,007826.jpg,500,375,chair,10,93,195,358,0.103000,0.737333,0.166000,0.434667
...,...,...,...,...,...,...,...,...,...,...,...,...
15655,000822.jpg,500,374,motorbike,231,353,168,323,0.584000,0.656417,0.244000,0.414439
15659,005796.jpg,375,500,person,45,294,57,500,0.452000,0.557000,0.664000,0.886000
15660,005796.jpg,375,500,tvmonitor,300,375,275,387,0.900000,0.662000,0.200000,0.224000
15661,002947.jpg,375,500,sofa,6,375,291,500,0.508000,0.791000,0.984000,0.418000


In [25]:
def label_encoding(x):
    labels = {'person':0, 'car':1, 'chair':2, 'bottle':3, 'pottedplant':4, 'bird':5, 'dog':6,
       'sofa':7, 'bicycle':8, 'horse':9, 'boat':10, 'motorbike':11, 'cat':12, 'tvmonitor':13,
       'cow':14, 'sheep':15, 'aeroplane':16, 'train':17, 'diningtable':18, 'bus':19}
    return labels[x]

In [26]:
train_df['id'] = train_df['name'].apply(label_encoding)
test_df['id'] = test_df['name'].apply(label_encoding)

/var/folders/69/hc5tv1pn547clcgc4pkpgsp80000gn/T/ipykernel_41591/3116627190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['id'] = train_df['name'].apply(label_encoding)
/var/folders/69/hc5tv1pn547clcgc4pkpgsp80000gn/T/ipykernel_41591/3116627190.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['id'] = test_df['name'].apply(label_encoding)


In [27]:
train_df

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h,id
0,007826.jpg,500,375,diningtable,80,320,217,273,0.400000,0.653333,0.480000,0.149333,18
1,007826.jpg,500,375,chair,197,257,193,326,0.454000,0.692000,0.120000,0.354667,2
2,007826.jpg,500,375,chair,139,185,184,231,0.324000,0.553333,0.092000,0.125333,2
3,007826.jpg,500,375,chair,258,312,180,314,0.570000,0.658667,0.108000,0.357333,2
4,007826.jpg,500,375,chair,10,93,195,358,0.103000,0.737333,0.166000,0.434667,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15655,000822.jpg,500,374,motorbike,231,353,168,323,0.584000,0.656417,0.244000,0.414439,11
15659,005796.jpg,375,500,person,45,294,57,500,0.452000,0.557000,0.664000,0.886000,0
15660,005796.jpg,375,500,tvmonitor,300,375,275,387,0.900000,0.662000,0.200000,0.224000,13
15661,002947.jpg,375,500,sofa,6,375,291,500,0.508000,0.791000,0.984000,0.418000,7


In [28]:
import os
from shutil import move

In [29]:
train_folder = 'data_images/train'
test_folder = 'data_images/test'


os.mkdir(train_folder)
os.mkdir(test_folder)

In [30]:
cols = ['filename','id','center_x','center_y', 'w', 'h']
groupby_obj_train = train_df[cols].groupby('filename')
groupby_obj_test = test_df[cols].groupby('filename')

In [31]:
#groupby_obj_train.get_group('000009.jpg').set_index('filename').to_csv('sample.txt',index=False,header=False)
# save each image in train/test folder and repective labels in .txt
def save_data(filename, folder_path, group_obj):
    # move image
    src = os.path.join('data_images',filename)
    dst = os.path.join(folder_path,filename)
    move(src,dst) # move image to the destination folder
    
    # save the labels
    text_filename = os.path.join(folder_path,
                                 os.path.splitext(filename)[0]+'.txt')
    group_obj.get_group(filename).set_index('filename').to_csv(text_filename,sep=' ',index=False,header=False)
    

In [32]:
filename_series = pd.Series(groupby_obj_train.groups.keys())

In [33]:
filename_series.apply(save_data,args=(train_folder,groupby_obj_train))

0       None
1       None
2       None
3       None
4       None
        ... 
4005    None
4006    None
4007    None
4008    None
4009    None
Length: 4010, dtype: object

In [34]:
filename_series_test = pd.Series(groupby_obj_test.groups.keys())
filename_series_test.apply(save_data,args=(test_folder,groupby_obj_test))

0       None
1       None
2       None
3       None
4       None
        ... 
997     None
998     None
999     None
1000    None
1001    None
Length: 1002, dtype: object